In [1]:
!pip3 install apache_beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import apache_beam as beam

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd drive/My\ Drive/Colab\ Notebooks/Cloud-AI-Analytics/Apache\ Beam\ -Python/data

/content/drive/My Drive/Colab Notebooks/Cloud-AI-Analytics/Apache Beam -Python/data


In [5]:
!ls

dept_data.txt  regular_filter.txt-00000-of-00001  Students_age.txt	students.txt
grocery.txt    result				  students_exclude.txt


In [6]:
!cat students.txt

Id,Student_name,City,Age,Marks,Result
1,vignesh,chn,27,15,FAIL
2,joey,us,51,20,FAIL
3,chandler,us,53,68,PASS
4,khaula,hyd,26,99,PASS
5,neethu,uae,27,100,PASS
6,sree,koc,25,27,FAIL
7,sai,mad,21,71,PASS
8,sabari,vel,25,75,PASS
9,tinkle,ker,27,9,FAIL
10,swati,ind,24,91,PASS


## **ParDo:**

•	ParDo is a Beam transform for generic parallel processing.

•	The ParDo processing paradigm is similar to the “Map” phase of a Map/Shuffle/Reduce-style algorithm: a ParDo transform considers each element in the input PCollection, performs some processing function (your user code) on that element, and emits zero, one, or multiple elements to an output PCollection.


In [9]:
class SplitRow(beam.DoFn):
  def process(self, element):
    return [element.split(',')]


class ComputeWordLengthFn(beam.DoFn):
  def process(self, element):
    return [len(element)]

In [10]:
with beam.Pipeline() as pipeline:
  input_data = (pipeline
                | "read from text">> beam.io.ReadFromText("students.txt", skip_header_lines= True)
                | "spliting the record" >> beam.ParDo(SplitRow()))

  count_data = (input_data
                |"filtering the data with PASS" >> beam.Filter(lambda record : record[5]=="FAIL"))

  word_lengths = (count_data
                 |"countof records" >> beam.ParDo(ComputeWordLengthFn())
                 |beam.Map(print))

  output_data = (count_data
                 | "Write to Text" >> beam.io.WriteToText("result/fail_data"))


6
6
6
6


In [13]:
with beam.Pipeline() as pipeline:
  input_data = (pipeline
                | "read from text">> beam.io.ReadFromText("students.txt", skip_header_lines= True)
                | "spliting the record" >> beam.ParDo(SplitRow()))

  count_data = (input_data
                |"filtering the data with PASS" >> beam.Filter(lambda record : record[5]=="PASS"))

  word_lengths = (count_data
                 |"countof records" >> beam.ParDo(ComputeWordLengthFn())
                 |beam.Map(print))

  output_data = (count_data
                 | "Write to Text" >> beam.io.WriteToText("result/pass_data"))


6
6
6
6
6
6


In [27]:
with beam.Pipeline() as pipeline:
  input_data = (pipeline
                | "read from text">> beam.io.ReadFromText("students.txt", skip_header_lines= True)
                | "spliting the record" >> beam.ParDo(SplitRow()))

  count_data = (input_data
                |"filtering the data with PASS" >> beam.Filter(lambda record : record[5]=="PASS"))

  output_data = (count_data
                 | "Write to Text" >> beam.io.WriteToText("result/pass_data"))

In [28]:
!ls result/

fail_data-00000-of-00001  pass_data-00000-of-00001  pass_students-00000-of-00001


In [24]:
!{('head -n 10 result/fail_data-00000-of-00001')}

['1', 'vignesh', 'chn', '27', '15', 'FAIL']
['2', 'joey', 'us', '51', '20', 'FAIL']
['6', 'sree', 'koc', '25', '27', 'FAIL']
['9', 'tinkle', 'ker', '27', '9', 'FAIL']


In [29]:
!{('head -n 10 result/pass_data-00000-of-00001')}

['3', 'chandler', 'us', '53', '68', 'PASS']
['4', 'khaula', 'hyd', '26', '99', 'PASS']
['5', 'neethu', 'uae', '27', '100', 'PASS']
['7', 'sai', 'mad', '21', '71', 'PASS']
['8', 'sabari', 'vel', '25', '75', 'PASS']
['10', 'swati', 'ind', '24', '91', 'PASS']


## **Keys:**

Takes a collection of key-value pairs and returns the key of each element.


In [15]:
with beam.Pipeline() as pipeline:
  icons = (
      pipeline
      | 'Garden plants' >> beam.Create([
          ('🍓', 'Strawberry'),
          ('🥕', 'Carrot'),
          ('🍆', 'Eggplant'),
          ('🍅', 'Tomato'),
          ('🥔', 'Potato'),
      ])
      | 'Keys' >> beam.Keys()
      | beam.Map(print))

🍓
🥕
🍆
🍅
🥔


## **Values:**

Takes a collection of key-value pairs, and returns the value of each element.



In [16]:
with beam.Pipeline() as pipeline:
  plants = (
      pipeline
      | 'Garden plants' >> beam.Create([
          ('🍓', 'Strawberry'),
          ('🥕', 'Carrot'),
          ('🍆', 'Eggplant'),
          ('🍅', 'Tomato'),
          ('🥔', 'Potato'),
      ])
      | 'Values' >> beam.Values()
      | beam.Map(print))

Strawberry
Carrot
Eggplant
Tomato
Potato


## **ToString**

Transforms every element in an input collection to a string. Any non-string element can be converted to a string using standard Python functions and methods. Many I/O transforms, such as textio.WriteToText, expect their input elements to be strings.



1. Key-value pairs to string
2. Elements to string
3. Iterables to string

In [30]:
with beam.Pipeline() as pipeline:
  plants = (
      pipeline
      | 'Garden plants' >> beam.Create([
          ('🍓', 'Strawberry'),
          ('🥕', 'Carrot'),
          ('🍆', 'Eggplant'),
          ('🍅', 'Tomato'),
          ('🥔', 'Potato'),
      ])
      | 'To string' >> beam.ToString.Kvs()  #Element() #Iterables()
      | beam.Map(print))

🍓,Strawberry
🥕,Carrot
🍆,Eggplant
🍅,Tomato
🥔,Potato


## **Kvswap :**

•	Takes a collection of key-value pairs and returns a collection of key-value pairs which has each key and value swapped.


In [31]:
with beam.Pipeline() as pipeline:
  plants = (
      pipeline
      | 'Garden plants' >> beam.Create([
          ('🍓', 'Strawberry'),
          ('🥕', 'Carrot'),
          ('🍆', 'Eggplant'),
          ('🍅', 'Tomato'),
          ('🥔', 'Potato'),
      ])
      | 'Key-Value swap' >> beam.KvSwap()
      | beam.Map(print))

('Strawberry', '🍓')
('Carrot', '🥕')
('Eggplant', '🍆')
('Tomato', '🍅')
('Potato', '🥔')
